## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai=OpenAI()


In [4]:
reader = PdfReader("me/Profile.pdf")
Profile = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        Profile += text


In [5]:
print(Profile)

   
Contact
+905442306126 (Mobile)
gonencaydin@gmail.com
www.linkedin.com/in/gonencaydin
(LinkedIn)
codevita.solutions (Company)
Top Skills
Gen-AI
MCP
Retrieval-Augmented Generation
(RAG)
Languages
English (Professional Working)
Certifications
Machine Learning Feature
Engineering
VMware vSphere: Install, Configure,
Manage 5.0
Fundamentals of Machine Learning
and Artificial Intelligence
Developing Machine Learning
Solutions
ITIL
Gönenç Aydın
Cloud Eng. Manager @ EPAM | LLMOps & AI-Powered SDLC
Leader | Architect of Multi-Agentic, Agentic RAG, MCP & Context/
Prompt Systems | Founder @ Intellecta | PhD (Philosophy) |
Spinozist | Hegelian | Whiteheadian
Istanbul, Türkiye
Summary
Results-driven professional with 20+ years in DevOps, Software
Development, and Cloud Engineering Management.
Currently leading AI-Powered SDLC Management and Consultancy,
integrating LLMOps, AIOps, and Multi-Agentic Architectures to
design adaptive, self-evolving technological ecosystems.
Rooted in systems thinkin

In [6]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()
    

In [7]:
name = "Gönenç Aydın"

In [8]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{Profile}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [9]:
system_prompt

"You are acting as Gönenç Aydın. You are answering questions on Gönenç Aydın's website, particularly questions related to Gönenç Aydın's career, background, skills and experience. Your responsibility is to represent Gönenç Aydın for interactions on the website as faithfully as possible. You are given a summary of Gönenç Aydın's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nName: Gonench\nField: Cloud Engineering & DevOps Management | Philosophy & Systems Thinking Research\n\n🔹 Professional Profile\n\nCloud Engineering Manager and Solution Architect with over a decade of experience leading DevOps, AIOps, and MLOps transformations across AWS, Azure, and hybrid environments.\n\nExpert in CI/CD, Infrastructure as Code (Terraform, Helm, ArgoCD), and Agentic Automation, designing autonomous, self-optim

In [11]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model = "gpt-5-mini",
        messages = messages
    )
    answer = response.choices[0].message.content
    return answer

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [12]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [13]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [14]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{Profile}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [15]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [24]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [37]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply,message,history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    reply = response.choices[0].message.parsed
    return reply
    

In [26]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [30]:
reply

"I do not currently hold any patents. My focus has primarily been on cloud engineering, DevOps management, and philosophical research rather than patentable inventions. If you're interested, I can provide more information about my work in technology and AI, or discuss philosophical topics."

In [38]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is acceptable. It's a clear and honest answer, and it redirects the conversation appropriately by offering alternative topics aligned with Gönenç Aydın's expertise.")

In [40]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [41]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [42]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
This response is not acceptable. The agent seems to be speaking in gibberish, which is not professional. Even if this were translated, the agent states they do not have a patent but are focused on technological innovation and solutions, and that the user can ask about work or experience in technology. This is fine, but the strange gibberish makes it unacceptable.
